In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import json
import random

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from essentia.standard import EasyLoader, FrameGenerator, MonoLoader

cwd = os.getcwd()
sys.path.append(os.path.dirname(cwd))

from lib.directories import DATASET_NAME, AUDIO_DIR, GT_PATH, FSD50K_DIR

import IPython.display as ipd

SAMPLE_RATE = 16000
TRIM_DUR = 30

FRAME_DUR = 0.96 #msec
#HOP_DUR = 0.1 #msec
FRAME_SIZE = int(SAMPLE_RATE*FRAME_DUR)

2024-02-04 18:02:06.043637: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
[   INFO   ] MusicExtractorSVM: no classifier models were configured by default
2024-02-04 18:02:06.458299: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-04 18:02:06.459824: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2024-02-04 18:02:06.481899: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-02-04 18:02:06.481918: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA dia

in_dir = '/home/ffont/fs_extractor_legacy_to_pca/out/'
out_dir = '/home/roguz/freesound/freesound-perceptual_similarity/data/embeddings/FSD50K.eval_audio/frederic-fs/'
import glob
json_paths = glob.glob(in_dir + '*.json')

for in_path in json_paths:
    out_path = out_dir + os.path.basename(in_path)
    audio_path = os.path.join(AUDIO_DIR, os.path.basename(in_path).replace('.json', '.wav'))
    with open(in_path) as in_f, open(out_path, 'w') as out_f:
        x = json.load(in_f)
        embed = {"audio_path": audio_path, "embeddings": x}
        json.dump(embed, out_f)


In [5]:
from lib.metrics import get_labels_of_fname, get_all_labels, find_indices_containing_label

In [6]:
df = pd.read_csv('../../data/pse_ucs_datasets-150k.csv')
df.head()

,file,top_level_category,sub_category,category_id,size,duration,fold,fname,labels
0,AIRBlow_Air-Can-Spray_PSE_BW-BD3_AEeY0.wav,AIR,BLOW,AIRBlow,80762,0.915669,train,AIRBlow_Air-Can-Spray_PSE_BW-BD3_AEeY0,AIR-BLOW
1,AIRBlow_Bottle-Wind-Eerie-Slowed_PSE_OCX_0tjTx...,AIR,BLOW,AIRBlow,2646494,30.005601,train,AIRBlow_Bottle-Wind-Eerie-Slowed_PSE_OCX_0tjTx,AIR-BLOW
2,AIRBlow_Party-Noise-Maker-Reedy-Hum-Plastic_PS...,AIR,BLOW,AIRBlow,217712,2.468390,train,AIRBlow_Party-Noise-Maker-Reedy-Hum-Plastic_PS...,AIR-BLOW
3,AIRBlow_Aerosol-Can-Spray_PSE_BW-BD3HL_As3yH.wav,AIR,BLOW,AIRBlow,100576,1.140317,train,AIRBlow_Aerosol-Can-Spray_PSE_BW-BD3HL_As3yH,AIR-BLOW
4,AIRBlow_Compressed-Air-Blown-through-Pipe-Hiss...,AIR,BLOW,AIRBlow,961044,10.896190,train,AIRBlow_Compressed-Air-Blown-through-Pipe-Hiss...,AIR-BLOW


In [7]:
get_labels_of_fname('AIRBlow_Aerosol-Can-Spray_PSE_BW-BD3HL_As3yH', df)

{'AIR-BLOW'}

In [8]:
all_labels = get_all_labels(df)

In [11]:
for label in all_labels:
    print(label, len(df[find_indices_containing_label(label, df)]))

LASERS-GUN 500
SCIFI-WEAPON 500
GUNS-CANNON 104
WIND-INTERIOR 335
GUNS-MISC 381
MAGIC-SPELL 365
WEAPONS-ARROW 500
CROWDS-CELEBRATION 5
TRAINS-DIESEL 500
MUSICAL-CHIME 500
DESIGNED-EERIE 500
WINDOWS-HARDWARE 36
BIRDS-WADING 183
OBJECTS-PACKAGING 56
VEHICLES-BRAKE 95
MECHANICAL-MISC 500
FIRE-CRACKLE 164
FOLEY-FEET 500
MECHANICAL-RATCHET 455
DOORS-HYDRAULIC & PNEUMATIC 61
HUMAN-MISC 40
WOOD-MOVEMENT 500
ROCKS-MOVEMENT 325
DOORS-HITECH 1
ICE-TONAL 7
FIREWORKS-RECREATIONAL 424
RAIN-GENERAL 305
VEHICLES-ELECTRIC 289
SPORTS-WINTER 500
COMMUNICATIONS-MISC 13
DIRT & SAND-DUST 4
GEOTHERMAL-FUMAROLE 3
ICE-BREAK 143
ROCKS-IMPACT 500
NATURAL DISASTER-AVALANCHE 12
MUSICAL-INSTRUMENT 500
ROPE-MOVEMENT 104
ANIMALS-CAT DOMESTIC 500
SCIFI-COMPUTER 2
AMBIENCE-DESIGNED 217
ELECTRICITY-ELECTROMAGNETIC 263
USER INTERFACE-MISC 253
DOORS-SLIDING 500
CREATURES-HUMANOID 113
WIND-TONAL 234
OBJECTS-KEYS 2
VEGETATION-TREE 233
CREATURES-DRAGON 2
CREATURES-INSECTOID 16
WATER-IMPACT 3
WATER-MOVEMENT 117
FIRE-BURNING 

In [11]:
df[find_indices_containing_label('AIR-BLOW', df)]

,file,top_level_category,sub_category,category_id,size,duration,fold,fname,labels
0,AIRBlow_Air-Can-Spray_PSE_BW-BD3_AEeY0.wav,AIR,BLOW,AIRBlow,80762,0.915669,train,AIRBlow_Air-Can-Spray_PSE_BW-BD3_AEeY0,AIR-BLOW
1,AIRBlow_Bottle-Wind-Eerie-Slowed_PSE_OCX_0tjTx...,AIR,BLOW,AIRBlow,2646494,30.005601,train,AIRBlow_Bottle-Wind-Eerie-Slowed_PSE_OCX_0tjTx,AIR-BLOW
2,AIRBlow_Party-Noise-Maker-Reedy-Hum-Plastic_PS...,AIR,BLOW,AIRBlow,217712,2.468390,train,AIRBlow_Party-Noise-Maker-Reedy-Hum-Plastic_PS...,AIR-BLOW
3,AIRBlow_Aerosol-Can-Spray_PSE_BW-BD3HL_As3yH.wav,AIR,BLOW,AIRBlow,100576,1.140317,train,AIRBlow_Aerosol-Can-Spray_PSE_BW-BD3HL_As3yH,AIR-BLOW
4,AIRBlow_Compressed-Air-Blown-through-Pipe-Hiss...,AIR,BLOW,AIRBlow,961044,10.896190,train,AIRBlow_Compressed-Air-Blown-through-Pipe-Hiss...,AIR-BLOW
...,...,...,...,...,...,...,...,...,...
102,AIRBlow_Spraycan-01_PSE_CLACK_K5PRM.wav,AIR,BLOW,AIRBlow,170278,1.930590,test,AIRBlow_Spraycan-01_PSE_CLACK_K5PRM,AIR-BLOW
103,AIRBlow_Do-Metal-Gaz-Pipe-Brake-And-Air-Pressu...,AIR,BLOW,AIRBlow,2646456,30.005170,test,AIRBlow_Do-Metal-Gaz-Pipe-Brake-And-Air-Pressu...,AIR-BLOW
104,AIRBlow_Bottle-Wind-Blow-Air-Across-Mouth-of-C...,AIR,BLOW,AIRBlow,1762622,19.984376,test,AIRBlow_Bottle-Wind-Blow-Air-Across-Mouth-of-C...,AIR-BLOW
105,AIRBlow_Shave-Cream-Spray-Full-Short1_PSE_BR-C...,AIR,BLOW,AIRBlow,53974,0.611950,test,AIRBlow_Shave-Cream-Spray-Full-Short1_PSE_BR-C...,AIR-BLOW


## pse_ucs_datasets creation

In [3]:
csv_path = '/home/ffont/research/pse_ucs_datasets/pse150k/dataset.csv'
df = pd.read_csv(csv_path)
df.head()

,file,top_level_category,sub_category,category_id,size,duration,fold
0,AIRBlow_Air-Can-Spray_PSE_BW-BD3_AEeY0.wav,AIR,BLOW,AIRBlow,80762,0.915669,train
1,AIRBlow_Bottle-Wind-Eerie-Slowed_PSE_OCX_0tjTx...,AIR,BLOW,AIRBlow,2646494,30.005601,train
2,AIRBlow_Party-Noise-Maker-Reedy-Hum-Plastic_PS...,AIR,BLOW,AIRBlow,217712,2.468390,train
3,AIRBlow_Aerosol-Can-Spray_PSE_BW-BD3HL_As3yH.wav,AIR,BLOW,AIRBlow,100576,1.140317,train
4,AIRBlow_Compressed-Air-Blown-through-Pipe-Hiss...,AIR,BLOW,AIRBlow,961044,10.896190,train


In [4]:
_df = df.copy()
_df["fname"] = _df['file'].apply(lambda x: os.path.splitext(x)[0])
_df['labels'] = _df['top_level_category'] + '-' + _df['sub_category']
_df.head()
_df.to_csv('pse_ucs_datasets-150k.csv', index=False)

In [8]:
df['duration'].describe()

count    143314.000000
mean         14.438669
std          12.444497
min           0.002313
25%           2.106429
50%          10.041259
75%          30.004354
max          30.009887
Name: duration, dtype: float64

In [9]:
df['top_level_category'].unique()

array(['AIR', 'AIRCRAFT', 'ALARMS', 'AMBIENCE', 'ANIMALS', 'ARCHIVED',
       'BEEPS', 'BELLS', 'BIRDS', 'BOATS', 'BULLETS', 'CARTOON',
       'CERAMICS', 'CHAINS', 'CHEMICALS', 'CLOCKS', 'CLOTH',
       'COMMUNICATIONS', 'COMPUTERS', 'CREATURES', 'CROWDS', 'DESIGNED',
       'DESTRUCTION', 'DIRT & SAND', 'DOORS', 'DRAWERS', 'ELECTRICITY',
       'EQUIPMENT', 'EXPLOSIONS', 'FARTS', 'FIGHT', 'FIRE', 'FIREWORKS',
       'FOLEY', 'FOOD & DRINK', 'FOOTSTEPS', 'GAMES', 'GEOTHERMAL',
       'GLASS', 'GORE', 'GUNS', 'HORNS', 'HUMAN', 'ICE', 'LASERS',
       'LEATHER', 'LIQUID & MUD', 'MACHINES', 'MAGIC', 'MECHANICAL',
       'METAL', 'MOTORS', 'MOVEMENT', 'MUSICAL', 'NATURAL DISASTER',
       'OBJECTS', 'PAPER', 'PLASTIC', 'RAIN', 'ROBOTS', 'ROCKS', 'ROPE',
       'RUBBER', 'SCIFI', 'SNOW', 'SPORTS', 'SWOOSHES', 'TOOLS', 'TOYS',
       'TRAINS', 'USER INTERFACE', 'VEGETATION', 'VEHICLES', 'VOICES',
       'WATER', 'WEAPONS', 'WEATHER', 'WHISTLES', 'WIND', 'WINDOWS',
       'WINGS', 'WOOD'], d

In [10]:
df['sub_category'].unique()

array(['BLOW', 'BURST', 'HISS', 'MISC', 'SUCTION', 'HELICOPTER', 'JET',
       'MECHANISM', 'MILITARY', 'PROP', 'RADIO CONTROLLED', 'ROCKET',
       'BELL', 'BUZZER', 'CLOCK', 'ELECTRONIC', 'SIREN', 'AMUSEMENT',
       'BIRDSONG', 'CELEBRATION', 'CONSTRUCTION', 'DESERT', 'DESIGNED',
       'EMERGENCY', 'FARM', 'FOREST', 'HISTORICAL', 'HITECH', 'HOSPITAL',
       'INDUSTRIAL', 'MARKET', 'NAUTICAL', 'OFFICE', 'PRISON',
       'PUBLIC PLACE', 'RELIGIOUS', 'RESTAURANT & BAR', 'ROOM TONE',
       'RURAL', 'SCHOOL', 'SEASIDE', 'SPORT', 'SUBURBAN', 'SWAMP',
       'TRAFFIC', 'TRANSPORTATION', 'TROPICAL', 'UNDERGROUND',
       'UNDERWATER', 'URBAN', 'WARFARE', 'AMPHIBIAN', 'AQUATIC', 'BAT',
       'CAT DOMESTIC', 'DOG', 'HORSE', 'INSECT', 'PRIMATE', 'REPTILE',
       'RODENT', 'WILD', 'IMPULSE RESPONSE', 'TEST TONE', 'APPLIANCE',
       'GENERAL', 'LOFI', 'MEDICAL', 'TIMER', 'VEHICLE', 'ANIMAL',
       'DOORBELL', 'GONG', 'HANDBELL', 'LARGE', 'BIRD OF PREY', 'CROW',
       'FOWL', 'SEA', 'SONG

In [15]:
top_bottom = {}
for top in df['top_level_category'].unique():
    bottom = set(df[df['top_level_category'] == top]['sub_category'].unique().tolist())
    top_bottom[top] = bottom

In [17]:
from itertools import combinations

In [20]:
for cat_a, cat_b in combinations(top_bottom.keys(), 2):
    inter = top_bottom[cat_a].intersection(top_bottom[cat_b])
    if len(inter) > 0:
        print(cat_a, cat_b, inter)

AIR AIRCRAFT {'MISC'}
AIR ALARMS {'MISC'}
AIR AMBIENCE {'MISC'}
AIR ANIMALS {'MISC'}
AIR BELLS {'MISC'}
AIR BIRDS {'MISC'}
AIR BOATS {'MISC'}
AIR BULLETS {'MISC'}
AIR CARTOON {'MISC'}
AIR CHAINS {'MISC'}
AIR CLOCKS {'MISC'}
AIR CLOTH {'MISC'}
AIR COMMUNICATIONS {'MISC'}
AIR COMPUTERS {'MISC'}
AIR CREATURES {'MISC'}
AIR CROWDS {'MISC'}
AIR DESIGNED {'MISC'}
AIR DESTRUCTION {'MISC'}
AIR DIRT & SAND {'MISC'}
AIR DOORS {'MISC'}
AIR DRAWERS {'MISC'}
AIR ELECTRICITY {'MISC'}
AIR EXPLOSIONS {'MISC'}
AIR FARTS {'MISC'}
AIR FIGHT {'MISC'}
AIR FIRE {'BURST', 'MISC'}
AIR FIREWORKS {'MISC'}
AIR FOLEY {'MISC'}
AIR FOOD & DRINK {'MISC'}
AIR GAMES {'MISC'}
AIR GLASS {'MISC'}
AIR GORE {'MISC'}
AIR GUNS {'MISC'}
AIR HORNS {'MISC'}
AIR HUMAN {'MISC'}
AIR ICE {'MISC'}
AIR LASERS {'MISC'}
AIR LEATHER {'MISC'}
AIR LIQUID & MUD {'SUCTION', 'MISC'}
AIR MACHINES {'MISC'}
AIR MAGIC {'MISC'}
AIR MECHANICAL {'MISC'}
AIR METAL {'MISC'}
AIR MOTORS {'MISC'}
AIR MUSICAL {'MISC'}
AIR NATURAL DISASTER {'MISC'}
AIR OBJ

In [11]:
len(df['category_id'].unique())

575

In [12]:
df['category_id'].unique()

array(['AIRBlow', 'AIRBrst', 'AIRHiss', 'AIRMisc', 'AIRSuck', 'AEROHeli',
       'AEROJet', 'AEROMech', 'AEROMil', 'AEROMisc', 'AEROProp',
       'AERORadio', 'AERORckt', 'ALRMBell', 'ALRMBuzr', 'ALRMClok',
       'ALRMElec', 'ALRMMisc', 'ALRMSirn', 'AMBAmus', 'AMBBird',
       'AMBCele', 'AMBCnst', 'AMBDsrt', 'AMBDsgn', 'AMBEmrg', 'AMBFarm',
       'AMBForst', 'AMBHist', 'AMBTech', 'AMBHosp', 'AMBInd', 'AMBMrkt',
       'AMBMisc', 'AMBNaut', 'AMBOffc', 'AMBPrisn', 'AMBPubl', 'AMBRlgn',
       'AMBRest', 'AMBRoom', 'AMBRurl', 'AMBSchl', 'AMBSea', 'AMBSprt',
       'AMBSubn', 'AMBSwmp', 'AMBTraf', 'AMBTran', 'AMBTrop', 'AMBUndr',
       'AMBUndwtr', 'AMBUrbn', 'AMBWar', 'ANMLAmph', 'ANMLAqua',
       'ANMLBat', 'ANMLCat', 'ANMLDog', 'ANMLFarm', 'ANMLHors',
       'ANMLInsc', 'ANMLMisc', 'ANMLPrim', 'ANMLRept', 'ANMLRdnt',
       'ANMLWild', 'IR', 'TEST', 'BEEPAppl', 'BEEP', 'BEEPLofi',
       'BEEPMed', 'BEEPTimer', 'BEEPVeh', 'BELLAnml', 'BELLDoor',
       'BELLGong', 'BELLHand', 'BELL